In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import optuna
import wandb
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [2]:
wandb.login(key="d12da696b882ebdf6b786d182d46febc1a77dcdb")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: diab55 (diab55-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# טען את הקובץ המעובד
df = pd.read_csv("/kaggle/input/train-data1/processed_train.csv")

# שנה את שם עמודת הטקסט
df = df.rename(columns={'fully_clean_text': 'Tweet'})

# קידוד התוויות ממחרוזות למספרים
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Sentiment'])

# שמור את המיפוי לצורך פיענוח בעתיד (אופציונלי)
for i, label in enumerate(label_encoder.classes_):
    print(f"{i} → {label}")

# חלוקה ל־Train / Test / Eval
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
test_df, eval_df = train_test_split(temp_df, test_size=1/3, random_state=42, stratify=temp_df['label'])

# שמירה של רק העמודות הדרושות למודל
train_df = train_df[['Tweet', 'label']]
test_df = test_df[['Tweet', 'label']]
eval_df = eval_df[['Tweet', 'label']]

# שמור את הקבצים החדשים
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)
eval_df.to_csv('eval_data.csv', index=False)


0 → Extremely Negative
1 → Extremely Positive
2 → Negative
3 → Neutral
4 → Positive


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification  # transformers = HF

# נטען את CT-BERT ממאגר HuggingFace
model_name = "digitalepidemiologylab/covid-twitter-bert"

# נטען את המודל עם מספר תוויות מתאים (5 – בהתאם ל־Sentiment שלך)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)
model  # הצגת מבנה המודל


config.json:   0%|          | 0.00/421 [00:00<?, ?B/s]

2025-08-05 21:02:45.355610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754427765.602308      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754427765.666931      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at digitalepidemiologylab/covid-twitter-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1

In [6]:
class TweetDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.texts = dataframe['Tweet'].tolist()
        self.labels = dataframe['label'].tolist() 
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]  # כבר int

        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=100,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [7]:
def early_stop_check(patience, best_val_accuracy, best_val_accuracy_epoch, current_val_accuracy, current_val_accuracy_epoch):
    early_stop_flag = False
    if current_val_accuracy > best_val_accuracy:
        best_val_accuracy = current_val_accuracy
        best_val_accuracy_epoch = current_val_accuracy_epoch
    else:
        if current_val_accuracy_epoch - best_val_accuracy_epoch > patience:
            early_stop_flag = True
    return best_val_accuracy, best_val_accuracy_epoch, early_stop_flag

In [8]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch

# ודא שיש לך את העמודה 'label' על בסיס 'Sentiment'
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Sentiment'])

# חישוב משקלי תוויות לפי הופעה בפועל
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df['label']),
    y=df['label']
)

# הפיכה לטנסור לשימוש בתוך CrossEntropyLoss
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# יצירת פונקציית הפסד הכוללת משקלים
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)


In [9]:
def train_model_with_hyperparams(model, train_loader, val_loader, optimizer, criterion, epochs, patience, trial):
    best_val_accuracy = 0.0
    best_val_accuracy_epoch = 0
    early_stop_flag = False
    best_model_state = None

    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0.0
        total_train_samples = 0
        correct_train_predictions = 0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item() * input_ids.size(0)
            total_train_samples += input_ids.size(0)
            correct_train_predictions += (logits.argmax(dim=1) == labels).sum().item()

        train_loss /= total_train_samples
        train_accuracy = correct_train_predictions / total_train_samples

        ### Validation ###
        model.eval()
        val_loss = 0.0
        total_val_samples = 0
        correct_val_predictions = 0
        all_val_labels = []
        all_val_preds = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                loss = criterion(logits, labels)

                val_loss += loss.item() * input_ids.size(0)
                total_val_samples += input_ids.size(0)
                correct_val_predictions += (logits.argmax(dim=1) == labels).sum().item()

                all_val_labels.extend(labels.cpu().numpy())
                all_val_preds.extend(logits.argmax(dim=1).cpu().numpy())

        val_loss /= total_val_samples
        val_accuracy = correct_val_predictions / total_val_samples

        # כאן שינוי חשוב: multi-class = average='weighted'
        val_precision = precision_score(all_val_labels, all_val_preds, average='weighted', zero_division=0)
        val_recall = recall_score(all_val_labels, all_val_preds, average='weighted', zero_division=0)
        val_f1 = f1_score(all_val_labels, all_val_preds, average='weighted', zero_division=0)

        best_val_accuracy, best_val_accuracy_epoch, early_stop_flag = early_stop_check(
            patience, best_val_accuracy, best_val_accuracy_epoch, val_accuracy, epoch)

        if val_accuracy == best_val_accuracy:
            best_model_state = model.state_dict()

        wandb.log({
            "Epoch": epoch,
            "Train Loss": train_loss,
            "Train Accuracy": train_accuracy,
            "Validation Loss": val_loss,
            "Validation Accuracy": val_accuracy,
            "Validation Precision": val_precision,
            "Validation Recall": val_recall,
            "Validation F1": val_f1
        })

        if early_stop_flag:
            break

    if best_model_state is not None:
        torch.save(best_model_state, f"best_model_trial_{trial.number}.pt")

    return best_val_accuracy


In [10]:
# Objective Function for Optuna
def objective(trial):
    # === Hyperparameter suggestions ===
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True)
    patience = trial.suggest_int("patience", 2, 5)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    num_layers = trial.suggest_int("num_layers", 1, 3)

    # === Tokenizer and Dataset ===
    tokenizer = AutoTokenizer.from_pretrained("digitalepidemiologylab/covid-twitter-bert")

    train_dataset = TweetDataset(train_df, tokenizer)
    val_dataset = TweetDataset(eval_df, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # === Load CT-BERT Model ===
    model = AutoModelForSequenceClassification.from_pretrained(
        "digitalepidemiologylab/covid-twitter-bert", num_labels=5
    ).to(device)

    # === Freeze all layers first ===
    for param in model.bert.parameters():
        param.requires_grad = False

    # === Unfreeze the last `num_layers` of encoder ===
    for param in model.bert.encoder.layer[-num_layers:].parameters():
        param.requires_grad = True

    # === Unfreeze the classification head ===
    for param in model.classifier.parameters():
        param.requires_grad = True

    # === Define loss with class weights ===
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

    # === Optimizer ===
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # === Initialize W&B for tracking ===
    wandb.init(
        project="ctbert-project",
        config={
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "patience": patience,
            "batch_size": batch_size,
            "num_layers": num_layers,
            "architecture": "CT-BERT",
            "dataset": "covid-tweets"
        },
        name=f"trial_{trial.number}"
    )

    # === Train and evaluate ===
    best_val_accuracy = train_model_with_hyperparams(
        model, train_loader, val_loader, optimizer, criterion, epochs=10,
        patience=patience, trial=trial
    )

    wandb.finish()
    return best_val_accuracy


In [ ]:
# Optuna Study
study = optuna.create_study(direction="maximize")  # אנו מעוניינים למקסם את ה-Validation Accuracy
study.optimize(objective, n_trials=10)  # הפעלת 10 ניסויים

[I 2025-08-05 21:03:39,318] A new study created in memory with name: no-name-03ec4ab9-b7f0-4ebc-96a2-5e54792918c7


vocab.txt: 0.00B [00:00, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at digitalepidemiologylab/covid-twitter-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,▁▂▃▃▄▅▆▆▇█
Train Accuracy,▁▃▄▅▆▆▇▇██
Train Loss,█▆▅▄▃▃▂▂▁▁
Validation Accuracy,▁▄▆▆▇▇▇▇██
Validation F1,▁▅▆▇▇▇▇▇██
Validation Loss,█▄▂▂▁▁▁▂▂▂
Validation Precision,▁▄▆▇▇▇▇▇██
Validation Recall,▁▄▆▆▇▇▇▇██
Epoch,10
Train Accuracy,0.79969
Train Loss,0.47467


[I 2025-08-05 22:10:23,284] Trial 0 finished with value: 0.6512420847540185 and parameters: {'learning_rate': 1.0650532870197462e-05, 'weight_decay': 8.174075923922814e-05, 'patience': 5, 'batch_size': 64, 'num_layers': 3}. Best is trial 0 with value: 0.6512420847540185.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at digitalepidemiologylab/covid-twitter-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

[I 2025-08-06 04:09:32,106] Trial 6 finished with value: 0.665854846566001 and parameters: {'learning_rate': 1.955683912476931e-05, 'weight_decay': 1.6290992478902544e-06, 'patience': 5, 'batch_size': 64, 'num_layers': 2}. Best is trial 1 with value: 0.701656113005358.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at digitalepidemiologylab/covid-twitter-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,▁▂▃▅▆▇█
Train Accuracy,▁▄▅▆▇██
Train Loss,█▅▄▃▂▁▁
Validation Accuracy,▁▃▃█▇▆▃
Validation F1,▂▂▁█▇▆▂
Validation Loss,█▃▃▁▃▃▅
Validation Precision,▁▄███▇▅
Validation Recall,▁▃▃█▇▆▃
Epoch,7
Train Accuracy,0.68996
Train Loss,0.7045


[I 2025-08-06 04:50:35,142] Trial 7 finished with value: 0.6310277642474428 and parameters: {'learning_rate': 0.00041522673002310846, 'weight_decay': 3.485129578494981e-06, 'patience': 2, 'batch_size': 64, 'num_layers': 1}. Best is trial 1 with value: 0.701656113005358.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at digitalepidemiologylab/covid-twitter-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,▁▂▃▃▄▅▆▆▇█
Train Accuracy,▁▃▅▆▆▇▇███
Train Loss,█▆▅▄▃▂▂▁▁▁
Validation Accuracy,▁▅▇▆▇▇▇███
Validation F1,▁▅▇▆▇▇▇███
Validation Loss,▃▁▁▂▃▃▆▆▆█
Validation Precision,▁▄▇▅▇▇▇███
Validation Recall,▁▅▇▆▇▇▇███
Epoch,10
Train Accuracy,0.94266
Train Loss,0.14846


[I 2025-08-06 05:57:21,297] Trial 8 finished with value: 0.709206039941549 and parameters: {'learning_rate': 0.0001137639693455489, 'weight_decay': 6.499757184807477e-05, 'patience': 4, 'batch_size': 64, 'num_layers': 3}. Best is trial 8 with value: 0.709206039941549.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at digitalepidemiologylab/covid-twitter-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
